In [1]:
# convert folder of csv format
#ID,LENGTH,DIR,A,B,ROADNAME,OTHERNAME,PROJECTNAM,YEAR,LEVEL_110,CLASS_110,LANES_110,ROW,S0,ALPHA,BETA,NMT_110,NTK_110,NBUS_110,COUNTY,MRTXFERPEN,ETAG
# "4",0.30,"2","17252","10118",台2,登輝大道,,"104","3","21","3",,"50",1.1507,3.7967,"0","0","0","2","0",0.0000
# to dat format
# tail.node	head.node	capacity..veh.h.	length..miles.	fftt.min.	Power	B	speed.limit..mph.
# 1	54632	99999	0	2.19029	0.9	3.6	NA

import pandas as pd

In [ ]:
#TRTS4S_Capacity.txt
#CLASS,L1,L2,L3,L4,L5,L6
#eliminate empty lines
#save as TRTS4S_Capacity.csv
# with open('TRTS4S_Capacity.txt', 'r') as f:
#     lines = f.readlines()

# with open('TRTS4S_Capacity.csv', 'w') as f:
#     for line in lines:
#         if line.strip():  # Check if the line is not empty
#             f.write(line)

In [2]:
cap_dict = {}
#input
# CLASS,L1,L2,L3,L4,L5,L6
# 1, 2300, 4300, 6300, 8300, 10300, 12300
#disct: (class,lanes)=capacity eg (39,6)=9420
with open('TRTS4S_Capacity.csv', 'r') as f:
    lines = f.readlines()
    for line in lines[1:]:  # Skip the header line
        parts = line.strip().split(',')
        class_num = int(parts[0])
        for i in range(1, len(parts)):
            lanes = i
            capacity = int(parts[i])
            cap_dict[(class_num, lanes)] = capacity

In [3]:
# Network

input_file = 'Y110_TRTS4S_Net_v1.csv'
output_file = 'network.dat'

df = pd.read_csv(input_file)
# if there is '', then replace with 0
df_out = pd.DataFrame(columns=['origin','dest','capacity','length','fft','alpha','beta','speedLimit'])
df.fillna(0, inplace=True)
rows = []

for index, row in df.iterrows():
    if row["LEVEL_110"] in [1,2,3,4,5,6,11] and row["CLASS_110"]: #not in [i for i in range(84, 99)]+[61]:
        # if DIR=1, then A is origin, B is dest
        # if DIR=-1, then A is dest, B is origin
        # if DIR=2, then the road is bidirectional, so we need to add two rows
        if row["DIR"] == 1 or row["DIR"] == 2:
            # print(row["ID"], row["DIR"], row["A"], row["B"])
            origin = row["A"]
            dest = row["B"]
        elif row["DIR"] == -1:
            # print(row["ID"], row["DIR"], row["A"], row["B"])
            origin = row["B"]
            dest = row["A"]
        capacity = cap_dict.get((row["CLASS_110"], row["LANES_110"]), 0)
        if row["CLASS_110"] == 99:
            capacity = 9999
        length = float(row["LENGTH"])  or 1000
        try:
            fft = length/float(row["S0"])*60
        except:
            # if ID is not 40438, then give S0=32
            # print(row["ID"], row["S0"])
            if row["ID"] != 40438:
                fft = length/32*60
            else:
                fft = 1000
        alpha = row["ALPHA"] or 0
        beta = row["BETA"] or 0
        speedLimit = 0
        if capacity == 0:
            continue
        rows.append({'origin':origin,'dest':dest,'capacity':capacity,'length':length,'fft':fft,'alpha':alpha,'beta':beta,'speedLimit':speedLimit})
        if row["DIR"] == 2:
            # add the reverse direction
            rows.append({'origin':dest,'dest':origin,'capacity':capacity,'length':length,'fft':fft,'alpha':alpha,'beta':beta,'speedLimit':speedLimit})

# tail.node	head.node	capacity..veh.h.		fftt.min.			speed.limit..mph.

df_out = pd.concat([df_out, pd.DataFrame(rows)], ignore_index=True)
df_out.to_csv(output_file, index=False, sep='\t')

In [ ]:
# # OD flow
# # read TRTS4S_Y110指派OD與交通分區.xlsx
# # sheet_name = '110年指派OD晨峰'
# # columm A,B,H,I as origin,dest,all,bus
# # demand as all+bus*1.8/19

# import pandas as pd
# import numpy as np

# input_file = 'TRTS4S_晨昏峰小時指派OD.xlsx'
# output_file = 'demand.dat'

# df = pd.read_excel(input_file, sheet_name='110年指派OD晨峰小時')
# df.fillna(0, inplace=True)
# rows = []

# print("Successfully read the file")

# count=0

# for index, row in df.iterrows():
#     origin = row["I"]
#     dest = row["J"]
#     all = row["ALL"]
#     bus = row["BUS"]
#     demand = all+bus*1.8/19
#     if origin <=691 and dest <=691:
#         rows.append({'origin':int(origin),'dest':int(dest),'demand':demand})
#     if count%10000==0:
#         print(count)
#     count+=1

# df_out = pd.DataFrame(rows)
# df_out.to_csv(output_file, index=False, sep='\t')


In [ ]:
# import pandas as pd
# import numpy as np

# input_file = 'TRTS4S_晨昏峰小時指派OD.xlsx'

# for i in [110,140]:
#     for j in ['晨峰','昏峰']:
#         df = pd.read_excel(input_file, sheet_name=f'{i}年指派OD{j}小時')
#         df.fillna(0, inplace=True)

#         #save as csv
#         output_file = f'{i}_{j}_demand.csv'
#         df.to_csv(output_file, index=False, sep=',')
#         print(f"Successfully saved {output_file}")
#
#6:55.8 complete

Successfully saved 110_晨峰_demand.csv
Successfully saved 110_昏峰_demand.csv
Successfully saved 140_晨峰_demand.csv
Successfully saved 140_昏峰_demand.csv


In [4]:
#read 110_晨峰_demand.csv to demand.dat

import pandas as pd
import numpy as np

df = pd.read_csv('110_晨峰_demand.csv')
df.fillna(0, inplace=True)
rows = []

count=0
for index, row in df.iterrows():
    origin = row["I"]
    dest = row["J"]
    all = row["ALL"]
    bus = row["BUS"]
    demand = all+bus*1.8/19
    out_of_cordon=[i for i in range(692,751)]+[783,784]
    if origin not in out_of_cordon and dest not in out_of_cordon:
        rows.append({'origin':int(origin),'dest':int(dest),'demand':demand})
    if count%10000==0:
        print(count)
    count+=1

df_out = pd.DataFrame(rows)
df_out.to_csv('demand.dat', index=False, sep='\t')

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
